<a href="https://colab.research.google.com/github/jeantirole/repo001/blob/master/0721_Jeju_append_ver04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

import pandas as pd
from pandas.io import gbq

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action = 'ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from os.path import join  

#MY_GOOGLE_DRIVE_PATH = 'My Drive/Data_Corona'
MY_GOOGLE_DRIVE_PATH = 'My Drive/Data_Jeju'

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print(PROJECT_PATH)

/content/drive/My Drive/Data_Jeju


In [4]:
%cd "{PROJECT_PATH}"
!ls

/content/drive/My Drive/Data_Jeju
201901-202003.csv  submission.csv


In [5]:
#데이터 로드
#코로나 확진자 데이터 로드 
P001= pd.read_csv('/content/drive/My Drive/Data_Corona/PatientInfo.csv')

In [6]:
#전국 신용카드 매출 데이터 로드
train= pd.read_csv('/content/drive/My Drive/Data_Jeju/201901-202003.csv')

In [7]:
import time 

In [8]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# # train load from v 
# %%time

# from google.cloud import bigquery
# from tabulate import tabulate
# import pandas as pd

# project_id = 'eric-dacon'
# client = bigquery.Client(project=project_id)

# train = client.query('''
#   SELECT 
#     * 
#   FROM `eric-dacon.jeju_data_ver1.201901_202003_train` 
#   ''').to_dataframe()

In [12]:
# # submission load from v 

# %%time
# project_id = 'eric-dacon'
# client = bigquery.Client(project=project_id)

# submission = client.query('''
#   SELECT 
#     * 
#   FROM `eric-dacon.jeju_data_ver1.submission` 
#   LIMIT 200000
#   ''').to_dataframe()

CPU times: user 54.1 ms, sys: 99 µs, total: 54.2 ms
Wall time: 1.6 s


In [9]:
whos

Variable               Type         Data/Info
---------------------------------------------
MY_GOOGLE_DRIVE_PATH   str          My Drive/Data_Jeju
P001                   DataFrame          patient_id     sex <...>n[5165 rows x 14 columns]
PROJECT_PATH           str          /content/drive/My Drive/Data_Jeju
ROOT                   str          /content/drive
auth                   module       <module 'google.colab.aut<...>es/google/colab/auth.py'>
drive                  module       <module 'google.colab.dri<...>s/google/colab/drive.py'>
gbq                    module       <module 'pandas.io.gbq' f<...>ckages/pandas/io/gbq.py'>
join                   function     <function join at 0x7f2f889a1f28>
mpl                    module       <module 'matplotlib' from<...>/matplotlib/__init__.py'>
pd                     module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
plt                    module       <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
sns               

In [10]:
#코로나 확진자 데이터 확인
P001

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,male,50s,Korea,Seoul,Gangseo-gu,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,male,30s,Korea,Seoul,Jungnang-gu,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,male,50s,Korea,Seoul,Jongno-gu,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,male,20s,Korea,Seoul,Mapo-gu,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,female,20s,Korea,Seoul,Seongbuk-gu,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5160,7000000015,female,30s,Korea,Jeju-do,Jeju-do,overseas inflow,NaN,25,NaN,2020-05-30,2020-06-13,NaN,released
5161,7000000016,NaN,NaN,Korea,Jeju-do,Jeju-do,overseas inflow,NaN,NaN,NaN,2020-06-16,2020-06-24,NaN,released
5162,7000000017,NaN,NaN,Bangladesh,Jeju-do,Jeju-do,overseas inflow,NaN,72,NaN,2020-06-18,NaN,NaN,isolated
5163,7000000018,NaN,NaN,Bangladesh,Jeju-do,Jeju-do,overseas inflow,NaN,NaN,NaN,2020-06-18,NaN,NaN,isolated


In [11]:
#확진자 데이터에서 년,월별 total_patient column 추가하기
result = {'year':[0], 
        'month':[0],
        'total_patient':[0],
        'CARD_SIDO_NM':[0]}
result = pd.DataFrame(result)

p001 = list(P001['province'].unique())

for i in p001: 
  A001 = P001[P001['province'] == i]
  a002 = A001['confirmed_date'].value_counts()
  a002 = pd.DataFrame(a002).reset_index()
  a002 = a002.rename({'index':'confirmed_date','confirmed_date':'total_patient'},axis='columns')
  #datetime 형태로 전환하기 
  a002['confirmed_date'] = pd.to_datetime(a002.confirmed_date, errors='coerce')
  a002['month'] = a002['confirmed_date'].dt.strftime('%m')
  a002['year'] = a002['confirmed_date'].dt.strftime('%Y')

  r001 = a002.groupby(['year','month'])['total_patient'].sum()
  r001 = pd.DataFrame(r001)
  r001 = r001.reset_index()
  r001['CARD_SIDO_NM'] = i
  result = result.append(r001)



In [12]:
#확진자 데이터에서 중복값 확인하기 
result[result['CARD_SIDO_NM']=='Gyeonggi-do']

,year,month,total_patient,CARD_SIDO_NM
0,2020,01,2,Gyeonggi-do
1,2020,02,82,Gyeonggi-do
2,2020,03,417,Gyeonggi-do
3,2020,04,177,Gyeonggi-do
4,2020,05,175,Gyeonggi-do
5,2020,06,355,Gyeonggi-do


In [13]:
#확진자 데이터 내에 시도 명 
result['CARD_SIDO_NM'].unique()

array([0, 'Seoul', 'Busan', 'Daegu', 'Gwangju', 'Incheon', 'Daejeon',
       'Gyeonggi-do', 'Sejong', 'Ulsan', 'Gangwon-do',
       'Chungcheongbuk-do', 'Chungcheongnam-do', 'Jeollabuk-do',
       'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do'],
      dtype=object)

In [14]:
#신용카드 매출 데이터 내에 시도 명 
train.CARD_SIDO_NM.unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

In [15]:
#확진자 데이터에 지역명을 한글로 교체 
result01 = result['CARD_SIDO_NM'].replace({'Seoul':'서울',
                        'Busan':'부산',
                        'Daegu':'대구',
                        'Gwangju':'광주',
                        'Incheon':'인천',
                        'Daejeon':'대전',
                        'Gyeonggi-do':'경기',
                        'Sejong':'세종', 
                        'Ulsan':'울산', 
                        'Gangwon-do':'강원',
                        'Chungcheongbuk-do':'충북',
                        'Chungcheongnam-do':'충남', 
                        'Jeollabuk-do':'전북',
                        'Jeollanam-do':'전남',
                        'Gyeongsangbuk-do':'경북', 
                        'Gyeongsangnam-do':'경남', 
                        'Jeju-do':'제주'}, inplace=True)

In [16]:
#한글 교체 확인
result[result.CARD_SIDO_NM=='경기']

,year,month,total_patient,CARD_SIDO_NM
0,2020,01,2,경기
1,2020,02,82,경기
2,2020,03,417,경기
3,2020,04,177,경기
4,2020,05,175,경기
5,2020,06,355,경기


In [17]:
# SIDO 가 0인 데이터 지우기
result01 = result[result['CARD_SIDO_NM'] != 0]
result01

,year,month,total_patient,CARD_SIDO_NM
0,2020,01,7,서울
1,2020,02,79,서울
2,2020,03,392,서울
3,2020,04,156,서울
4,2020,05,228,서울
...,...,...,...,...
0,2020,02,2,제주
1,2020,03,7,제주
2,2020,04,4,제주
3,2020,05,2,제주


In [18]:
# datetime 형태로 전환하기 

In [19]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [20]:
data = train

In [21]:
result01

,year,month,total_patient,CARD_SIDO_NM
0,2020,01,7,서울
1,2020,02,79,서울
2,2020,03,392,서울
3,2020,04,156,서울
4,2020,05,228,서울
...,...,...,...,...
0,2020,02,2,제주
1,2020,03,7,제주
2,2020,04,4,제주
3,2020,05,2,제주


In [22]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [23]:
# 날짜 처리
#data = pd.read_csv('data/201901-202003.csv')
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [39]:
result01

,year,month,total_patient,CARD_SIDO_NM
0,2020,1,7,서울
1,2020,2,79,서울
2,2020,3,392,서울
3,2020,4,156,서울
4,2020,5,228,서울
...,...,...,...,...
0,2020,2,2,제주
1,2020,3,7,제주
2,2020,4,4,제주
3,2020,5,2,제주


In [24]:
# 날짜 처리 코로나 확진자 데이터
result01 = result01.astype({'month':int, 
                            'year':int,})

In [25]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [26]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [30]:
# 여기서 data로 merge 하지 말고, encoding 끝난 df 로 merge 해야합니다. 

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057394 entries, 0 to 1057393
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   CARD_SIDO_NM  1057394 non-null  object
 1   STD_CLSS_NM   1057394 non-null  object
 2   HOM_SIDO_NM   1057394 non-null  object
 3   AGE           1057394 non-null  object
 4   SEX_CTGO_CD   1057394 non-null  int64 
 5   FLC           1057394 non-null  int64 
 6   year          1057394 non-null  int64 
 7   month         1057394 non-null  int64 
 8   CSTMR_CNT     1057394 non-null  int64 
 9   AMT           1057394 non-null  int64 
 10  CNT           1057394 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 88.7+ MB


In [31]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057394 entries, 0 to 1057393
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype
---  ------        --------------    -----
 0   CARD_SIDO_NM  1057394 non-null  int64
 1   STD_CLSS_NM   1057394 non-null  int64
 2   HOM_SIDO_NM   1057394 non-null  int64
 3   AGE           1057394 non-null  int64
 4   SEX_CTGO_CD   1057394 non-null  int64
 5   FLC           1057394 non-null  int64
 6   year          1057394 non-null  int64
 7   month         1057394 non-null  int64
 8   CSTMR_CNT     1057394 non-null  int64
 9   AMT           1057394 non-null  int64
 10  CNT           1057394 non-null  int64
dtypes: int64(11)
memory usage: 88.7 MB


In [29]:
result01.dtypes

year              int64
month             int64
total_patient     int64
CARD_SIDO_NM     object
dtype: object

In [34]:
#신용카드 매출 데이터에 확진자 데이터 합치기
data01 = pd.merge(df, result01, how='left')

In [35]:
data01.info()
#병합하면서 추가된 total_patient column 은 2019년도에 값이 모두 결측치로 존재한다

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057394 entries, 0 to 1057393
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   CARD_SIDO_NM   1057394 non-null  object 
 1   STD_CLSS_NM    1057394 non-null  object 
 2   HOM_SIDO_NM    1057394 non-null  object 
 3   AGE            1057394 non-null  object 
 4   SEX_CTGO_CD    1057394 non-null  int64  
 5   FLC            1057394 non-null  int64  
 6   year           1057394 non-null  int64  
 7   month          1057394 non-null  int64  
 8   CSTMR_CNT      1057394 non-null  int64  
 9   AMT            1057394 non-null  int64  
 10  CNT            1057394 non-null  int64  
 11  total_patient  138760 non-null   float64
dtypes: float64(1), int64(7), object(4)
memory usage: 104.9+ MB


In [46]:
# 병합된 데이터의 인코딩 

dtypes = data01.dtypes
encoders = {}
for column in data01.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(data01[column])
        encoders[column] = encoder
        
df_num = data01.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(data01[column])


In [48]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057394 entries, 0 to 1057393
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   CARD_SIDO_NM   1057394 non-null  int64  
 1   STD_CLSS_NM    1057394 non-null  int64  
 2   HOM_SIDO_NM    1057394 non-null  int64  
 3   AGE            1057394 non-null  int64  
 4   SEX_CTGO_CD    1057394 non-null  int64  
 5   FLC            1057394 non-null  int64  
 6   year           1057394 non-null  int64  
 7   month          1057394 non-null  int64  
 8   CSTMR_CNT      1057394 non-null  int64  
 9   AMT            1057394 non-null  int64  
 10  CNT            1057394 non-null  int64  
 11  total_patient  138760 non-null   float64
dtypes: float64(1), int64(11)
memory usage: 104.9 MB


In [58]:
df_num01 = df_num.copy() #df_0 = df.fillna(0)

In [59]:
#df_num01.total_patient = df_num01.total_patient.fillna(0)
df_num01.total_patient = df_num01.total_patient.fillna(0)

In [64]:
df_num01.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT,total_patient
0,0,0,0,1,1,1,2019,1,4,311200,4,0.0
1,0,0,0,1,1,1,2019,2,3,605000,3,0.0
2,0,0,0,1,1,1,2019,6,3,139000,3,0.0
3,0,0,0,1,1,1,2019,8,3,27500,3,0.0
4,0,0,0,1,1,1,2019,9,3,395500,3,0.0


In [65]:
df_num01 = df_num01.astype({'total_patient':int})

In [37]:
from sklearn.ensemble import GradientBoostingRegressor

In [70]:
# 변수 선택 및 모델 구축
# feature, target 설정
#train_num = df_num.sample(frac=1, random_state=0) => 원래 코드에서는 df_num 을 sampling 해서, train_num 을 뽑았다. 하지만 df_num 의 경우 전체 column 이 int64 로 되어있기 때문에, 그러한 형태가 현재의 lgbm 모델에 적합한 지 의문이 든다. 따라서, df_num 을 쓰지 않고, df 와 result01 을 merge 한 데이터를 쓰도록 하겠다. 
train_num = df_num01.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [71]:
gbm = GradientBoostingRegressor(n_estimators=4000);
gbm.fit(train_features, train_target)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=4000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
from itertools import product

# 예측 템플릿 만들기 1,2 두가지로 나눠서, 하나는 total_patient 를 넣은것 하나는 안 넣은것 
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
Total_PA      = df_num['total_patient'].unique()

years         = [2020]
months        = [4, 7]

comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs,HOM_SIDO_NMs, AGEs, SEX_CTGO_CDs, FLCs, years, months,Total_PA]
temp = np.array(list(product(*comb_list)))
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [53]:
# 예측
pred = gbm.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

KeyError: ignored

In [39]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [39]:
print(temp.head())

In [ ]:
# submission load from v 

%%time
project_id = 'eric-dacon'
client = bigquery.Client(project=project_id)

submission = client.query('''
  SELECT 
    * 
  FROM `eric-dacon.jeju_data_ver1.submission` 
  LIMIT 200000
  ''').to_dataframe()

In [39]:
submission = client.query('''
  SELECT 
    * 
  FROM `eric-dacon.jeju_data_ver1.submission` 
  ''').to_dataframe()

In [39]:
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission['AMT'] = submission['AMT'].fillna(0)
print(submission.head())

In [ ]:
submission.to_csv('submission.csv', encoding='utf-8-sig', index=False)